<a href="https://colab.research.google.com/github/nitin-barthwal/CNN1D/blob/master/CNN_1D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:

import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import plotly
import plotly.express as px
from datetime import datetime,timedelta
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.metrics import mean_squared_error,  mean_absolute_error
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Flatten, Dropout, Reshape
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.utils import to_categorical
#from keras.utils import plot_model
from keras.utils.vis_utils import plot_model
from keras import backend
import keras
#from keras.layers.core import Dense, Activation, Dropout


from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import warnings
warnings.filterwarnings("ignore")

import plotly.graph_objs as go
plotly.offline.init_notebook_mode(connected=True)

In [29]:
url = 'https://raw.githubusercontent.com/nitin-barthwal/CNN1D/master/dataset.csv'
rescaled = pd.read_csv(url)
print('Shape : ',rescaled.shape)
rescaled.sample(10)

Shape :  (177126, 2)


,Input,Output
146532,0.014218,0.081833
92497,0.013744,0.047393
69124,0.000000,0.055608
25474,0.000000,0.065719
172851,0.013586,0.066667
110577,0.000000,0.395261
1870,0.000000,0.043444
67892,0.014534,0.076145
143887,0.000000,0.051817
82644,0.038547,0.091153


In [30]:

def Reshape_data(df, time_steps, step):
    '''
    time_steps is the number of observations that I have in a sample 
    steps is the number with which I am shifting or advancing to create a new sample. 
    for eg if data is [ 1,2,3,4,5,6,7,8,9,10,11,12] with time steps = 5 and step=3 then 
    sample 1: [1,2,3,4,5]
    sample 2: [4,5,6,7,8] 
    sample 3: [7,8,9,10,11]
    '''
        
    N_FEATURES = 1   # As i have only one input feature
    segments = []
    output_segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
        sensor_load = df['Input'].values[i: i + time_steps]
        segments.append([sensor_load])
        output= df['Output'].values[i: i + time_steps]
        output_segments.append([output])
    # Reshaping
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, N_FEATURES)
    output_segments = np.asarray(output_segments, dtype= np.float32).reshape(-1, time_steps, N_FEATURES)
    return reshaped_segments, output_segments

TIME_PERIODS= 600

STEP_DISTANCE = 1 # Shifting just 1 observation

x_train,y_train = Reshape_data(rescaled, TIME_PERIODS, STEP_DISTANCE )
print('x_train.shape : ',x_train.shape)
print('y_train.shape : ',y_train.shape)

x_train.shape :  (176526, 600, 1)
y_train.shape :  (176526, 600, 1)


In [31]:
x_train

array([[[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.        ],
        [0.        ],
        [0.        ],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       ...,

       [[0.01406003],
        [0.01406003],
        [0.01406003],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.01406003],
        [0.01406003],
        [0.01406003],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]],

       [[0.01406003],
        [0.01406003],
        [0.01406003],
        ...,
        [0.        ],
        [0.        ],
        [0.        ]]], dtype=float32)

In [32]:
y_train

array([[[0.0436019 ],
        [0.0436019 ],
        [0.04375987],
        ...,
        [0.04881516],
        [0.04881516],
        [0.04818325]],

       [[0.0436019 ],
        [0.04375987],
        [0.04423381],
        ...,
        [0.04881516],
        [0.04818325],
        [0.04818325]],

       [[0.04375987],
        [0.04423381],
        [0.04375987],
        ...,
        [0.04818325],
        [0.04818325],
        [0.04818325]],

       ...,

       [[0.06018957],
        [0.06018957],
        [0.06097946],
        ...,
        [0.10537125],
        [0.10837283],
        [0.10837283]],

       [[0.06018957],
        [0.06097946],
        [0.06097946],
        ...,
        [0.10837283],
        [0.10837283],
        [0.10537125]],

       [[0.06097946],
        [0.06097946],
        [0.06050553],
        ...,
        [0.10837283],
        [0.10537125],
        [0.10537125]]], dtype=float32)

In [33]:
num_time_periods, n_features = x_train.shape[1], x_train.shape[2]

input_shape = (num_time_periods*n_features)
x_train = x_train.reshape(x_train.shape[0], input_shape)

print('x_train shape:', x_train.shape)
print('input_shape:', input_shape)

x_train shape: (176526, 600)
input_shape: 600


In [34]:
x_train

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.01406003, 0.01406003, 0.01406003, ..., 0.        , 0.        ,
        0.        ],
       [0.01406003, 0.01406003, 0.01406003, ..., 0.        , 0.        ,
        0.        ],
       [0.01406003, 0.01406003, 0.01406003, ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [35]:
 
def create_model(n_timesteps, n_features,n_outputs):
    '''Creates and returns the ShortSeq2Point Network
    '''
    model = Sequential()

    # 1D Conv
    model.add(Conv1D(filters=30, kernel_size=10, activation='relu', input_shape=(n_timesteps, n_features), padding="same", strides=1))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=30, kernel_size=8, activation='relu', padding="same", strides=1))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=40, kernel_size=6, activation='relu', padding="same", strides=1))
    model.add(Dropout(0.5))
    model.add(Conv1D(filters=50, kernel_size=5, activation='relu', padding="same", strides=1))
    model.add(Dropout(0.5))
    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_outputs, activation='linear'))

    model.compile(loss='mse', optimizer='adam')
    #plot_model(model, to_file='model.png', show_shapes=True)

    return model

#n_timesteps, n_features, n_outputs = x_train.shape[1], x_train.shape[2], y_train.shape[1]
n_outputs=y_train.shape[1]
# create model
model = create_model(num_time_periods, n_features,n_outputs)
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_18 (Conv1D)           (None, 600, 30)           330       
_________________________________________________________________
dropout_20 (Dropout)         (None, 600, 30)           0         
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 600, 30)           7230      
_________________________________________________________________
dropout_21 (Dropout)         (None, 600, 30)           0         
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 600, 40)           7240      
_________________________________________________________________
dropout_22 (Dropout)         (None, 600, 40)           0         
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 600, 50)          

In [36]:
callbacks_list = [
    keras.callbacks.ModelCheckpoint(
        filepath='best_model.{epoch:02d}-{val_loss:.2f}.h5',
        monitor='val_loss', save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='acc', patience=1)
]

model.compile(loss='mse',
                optimizer='adam', metrics=['mse'])

# Hyper-parameters
BATCH_SIZE = 400
EPOCHS = 1

# Enable validation to use ModelCheckpoint and EarlyStopping callbacks.
history = model.fit(x_train,
                      y_train,
                      batch_size=BATCH_SIZE,
                      epochs=EPOCHS,
                      #callbacks=callbacks_list,
                      validation_split=0.2,
                      verbose=1)

ValueError: ignored